In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

from tensorflow.keras.layers import Input, Activation, Dense, Flatten, RepeatVector, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import time
from datetime import datetime

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'dataset/train',
        target_size=(88, 88),
        batch_size=70,
        color_mode="grayscale",
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        'dataset/valid',
        target_size=(88, 88),    
        batch_size=80,
        color_mode="grayscale",
        class_mode='categorical')

Found 94500 images belonging to 11 classes.
Found 18800 images belonging to 11 classes.


In [3]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(88,88,1), kernel_initializer='he_normal'))
model.add(Conv2D(filters = 32, kernel_size = (1,1), strides = (1,1), padding = 'valid', kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(88,88,1), kernel_initializer='he_normal'))
model.add(Conv2D(filters = 32, kernel_size = (1,1), strides = (1,1), padding = 'valid', kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.7))

model.add(Conv2D(64, kernel_size=(3, 3), input_shape=(88,88,1), kernel_initializer='he_normal'))
model.add(Conv2D(filters = 64, kernel_size = (1,1), strides = (1,1), padding = 'valid', kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(64, kernel_size=(3, 3), input_shape=(88,88,1), kernel_initializer='he_normal'))
model.add(Conv2D(filters = 64, kernel_size = (1,1), strides = (1,1), padding = 'valid', kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(256, kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.7))

model.add(Dense(11, activation='softmax', kernel_initializer='he_normal'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 86, 86, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 86, 86, 32)        1056      
_________________________________________________________________
batch_normalization (BatchNo (None, 86, 86, 32)        128       
_________________________________________________________________
activation (Activation)      (None, 86, 86, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 84, 84, 32)        9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 84, 84, 32)        1056      
_________________________________________________________________
batch_normalization_1 (Batch (None, 84, 84, 32)        1

In [4]:
early_stopping = EarlyStopping(monitor='val_loss',mode='min', patience=10)
logdir="logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=logdir)



In [5]:
model.fit_generator(
        train_generator,
        steps_per_epoch=1350,
        epochs=20,
        validation_data=test_generator,
        validation_steps=235,
        callbacks=[early_stopping, tensorboard])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
   1/1350 [..............................] - ETA: 0s - accuracy: 0.0143 - loss: 4.1189WARNING:tensorflow:From /home/hagler/anaconda3/envs/capstone/lib/python3.5/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
1350/1350 [==============================] - 1440s 1s/step - accuracy: 0.7947 - loss: 0.5679 - val_loss: 1.5803 - val_accuracy: 0.6002
Epoch 2/20
1350/1350 [==============================] - 171s 126ms/step - accuracy: 0.9241 - loss: 0.2253 - val_loss: 1.3993 - val_accuracy: 0.6046
Epoch 3/20
1350/1350 [==============================] - 171s 127ms/step - accuracy: 0.9522 - loss: 0.1418 - val_loss: 0.3394 - val_accuracy: 0.8807
Epoch 4/20
1350/1350 [==============================] - 171s 127ms/step - accuracy: 0.9670 -

In [11]:
print("-- Evaluate --")
scores = model.evaluate_generator(test_generator, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

-- Evaluate --
accuracy: 98.50%


In [12]:
print("-- Predict --")
output = model.predict_generator(test_generator, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)

-- Predict --
Instructions for updating:
Please use Model.predict, which supports generators.
{'2': 3, '5': 6, '9': 10, '11': 2, '6': 7, '10': 1, '8': 9, '4': 5, '3': 4, '7': 8, '1': 0}
[[0.000 0.000 0.000 ... 0.000 0.000 0.000]
 [0.000 0.000 0.000 ... 0.000 0.000 0.000]
 [0.000 0.000 0.000 ... 0.000 1.000 0.000]
 ...
 [0.000 0.000 0.000 ... 0.000 0.000 0.000]
 [0.000 0.000 0.000 ... 0.000 0.000 0.000]
 [0.000 0.000 1.000 ... 0.000 0.000 0.000]]


In [13]:
model.save("lego_sorter_v7.h5")